- choropleth map for all the teams and their favorite (worst) ground
- best openers and their help in win
- best finishers and their help in win
- bowlers and batsman pair to watch
- map of most destructive batsman
- out of 100 balls what happens on them

In [1]:
import pandas as pd
from collections import Counter

In [2]:
df = pd.read_csv('./datasets/matches.dmp')

In [3]:
df.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2008,Bangalore,2008-04-18,Kolkata Knight Riders,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Kolkata Knight Riders,140,0,BB McCullum,M Chinnaswamy Stadium,Asad Rauf,RE Koertzen,NaN
1,2,2008,Chandigarh,2008-04-19,Chennai Super Kings,Kings XI Punjab,Chennai Super Kings,bat,normal,0,Chennai Super Kings,33,0,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",MR Benson,SL Shastri,NaN
2,3,2008,Delhi,2008-04-19,Rajasthan Royals,Delhi Daredevils,Rajasthan Royals,bat,normal,0,Delhi Daredevils,0,9,MF Maharoof,Feroz Shah Kotla,Aleem Dar,GA Pratapkumar,NaN
3,4,2008,Mumbai,2008-04-20,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,normal,0,Royal Challengers Bangalore,0,5,MV Boucher,Wankhede Stadium,SJ Davis,DJ Harper,NaN
4,5,2008,Kolkata,2008-04-20,Deccan Chargers,Kolkata Knight Riders,Deccan Chargers,bat,normal,0,Kolkata Knight Riders,0,5,DJ Hussey,Eden Gardens,BF Bowden,K Hariharan,NaN


### Teams

In [4]:
teams = list(df['team1'].unique())
teams

['Kolkata Knight Riders',
 'Chennai Super Kings',
 'Rajasthan Royals',
 'Mumbai Indians',
 'Deccan Chargers',
 'Kings XI Punjab',
 'Royal Challengers Bangalore',
 'Delhi Daredevils',
 'Kochi Tuskers Kerala',
 'Pune Warriors',
 'Sunrisers Hyderabad',
 'Rising Pune Supergiants',
 'Gujarat Lions']

### Ground Map Processing

### Grounds

In [5]:
grounds = list(df['venue'].unique())
grounds

['M Chinnaswamy Stadium',
 'Punjab Cricket Association Stadium, Mohali',
 'Feroz Shah Kotla',
 'Wankhede Stadium',
 'Eden Gardens',
 'Sawai Mansingh Stadium',
 'Rajiv Gandhi International Stadium, Uppal',
 'MA Chidambaram Stadium, Chepauk',
 'Dr DY Patil Sports Academy',
 'Newlands',
 "St George's Park",
 'Kingsmead',
 'SuperSport Park',
 'Buffalo Park',
 'New Wanderers Stadium',
 'De Beers Diamond Oval',
 'OUTsurance Oval',
 'Brabourne Stadium',
 'Sardar Patel Stadium, Motera',
 'Barabati Stadium',
 'Vidarbha Cricket Association Stadium, Jamtha',
 'Himachal Pradesh Cricket Association Stadium',
 'Nehru Stadium',
 'Holkar Cricket Stadium',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'Subrata Roy Sahara Stadium',
 'Shaheed Veer Narayan Singh International Stadium',
 'JSCA International Stadium Complex',
 'Sheikh Zayed Stadium',
 'Sharjah Cricket Stadium',
 'Dubai International Cricket Stadium',
 'Maharashtra Cricket Association Stadium',
 'Punjab Cricket Association IS Bind

In [45]:
non_indian_grounds = [
     'Newlands',
     "St George's Park",
     'Kingsmead',
     'SuperSport Park',
     'Buffalo Park',
     'New Wanderers Stadium',
     'De Beers Diamond Oval',
     'OUTsurance Oval',
     'Sheikh Zayed Stadium',
     'Sharjah Cricket Stadium',
     'Dubai International Cricket Stadium',
]

def is_indian_ground(ground_name):
    return False if ground_name in non_indian_grounds else True

In [50]:
def get_grounds_results(team_name):
    # append the team1/team2 appearance
    df1 = df[df['team1'] == team_name]
    df2 = df[df['team2'] == team_name]
    _df = df1.append(df2)
    
    # win and lose for each ground
    _won = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] == team_name]['venue'])))
    _lost = Counter(filter(lambda g: is_indian_ground(g), list(_df[_df['winner'] != team_name]['venue'])))
    
    # total grounds the team has played on
    _grounds = set(_won).union(set(_lost))
    
    _matches = []
    
    # Mohali ground has multiple names in dataset
    mohali_v1_name = "Punjab Cricket Association Stadium, Mohali"
    mohali_v2_name = "Punjab Cricket Association IS Bindra Stadium, Mohali"
    mohali_v2_won, mohali_v2_lost = _won.get(mohali_v2_name, 0), _lost.get(mohali_v2_name, 0)
    
    # iterate over all grounds
    for g in list(_grounds):
        # ignore the mohali grounds
        if g not in [mohali_v1_name, mohali_v2_name]:
            _matches.append({
                'ground': g,
                'won': _won.get(g, 0),
                'lost': _lost.get(g, 0),
            })  
            
    # unify the mohali results
    _matches.append({
        'ground': mohali_v1_name,
        'won': _won.get(mohali_v1_name, 0) + (mohali_v2_won),
        'lost': _lost.get(mohali_v1_name, 0) + (mohali_v2_lost),
    })
    
    result = {
        team_name: _matches
    }
    
    return result

In [51]:
for t in teams:
    print('--------------------\n', t)
    print(get_grounds_results(t))

--------------------
 Kolkata Knight Riders
{'Kolkata Knight Riders': [{'ground': 'M Chinnaswamy Stadium', 'won': 4, 'lost': 4}, {'ground': 'Barabati Stadium', 'won': 3, 'lost': 0}, {'ground': 'Eden Gardens', 'won': 33, 'lost': 18}, {'ground': 'JSCA International Stadium Complex', 'won': 1, 'lost': 2}, {'ground': 'Sardar Patel Stadium, Motera', 'won': 0, 'lost': 2}, {'ground': 'Wankhede Stadium', 'won': 1, 'lost': 5}, {'ground': 'Rajiv Gandhi International Stadium, Uppal', 'won': 4, 'lost': 1}, {'ground': 'Maharashtra Cricket Association Stadium', 'won': 2, 'lost': 0}, {'ground': 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium', 'won': 0, 'lost': 1}, {'ground': 'Brabourne Stadium', 'won': 0, 'lost': 2}, {'ground': 'Green Park', 'won': 0, 'lost': 1}, {'ground': 'Shaheed Veer Narayan Singh International Stadium', 'won': 0, 'lost': 1}, {'ground': 'Nehru Stadium', 'won': 0, 'lost': 1}, {'ground': 'Sawai Mansingh Stadium', 'won': 1, 'lost': 3}, {'ground': 'Dr DY Patil Sports Academy', 

### Toss Effect Processing

In [57]:
df.shape

(577, 18)

In [56]:
df[df['toss_winner'] == df['winner']].shape

(291, 18)

In [59]:
291/577

0.5043327556325823

### Slope Chart Processing

In [60]:
delivery_df = pd.read_csv('./datasets/deliveries.dmp')

In [61]:
delivery_df.shape

(136598, 21)

In [63]:
delivery_df.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder'],
      dtype='object')

In [67]:
delivery_df[delivery_df['batting_team'] == 'Kolkata Knight Riders'].tail(20)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
136086,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,17,5,R Sathish,MK Pandey,Mustafizur Rahman,0,...,0,1,0,0,0,1,1,NaN,NaN,NaN
136087,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,17,6,MK Pandey,R Sathish,Mustafizur Rahman,0,...,0,0,0,0,4,0,4,NaN,NaN,NaN
136088,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,1,R Sathish,MK Pandey,B Kumar,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
136089,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,2,MK Pandey,R Sathish,B Kumar,0,...,0,0,0,0,0,0,0,MK Pandey,caught,DJ Hooda
136090,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,3,R Sathish,JO Holder,B Kumar,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
136091,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,4,R Sathish,JO Holder,B Kumar,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
136092,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,5,R Sathish,JO Holder,B Kumar,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
136093,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,18,6,JO Holder,R Sathish,B Kumar,0,...,0,0,0,0,2,0,2,NaN,NaN,NaN
136094,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,19,1,R Sathish,JO Holder,Mustafizur Rahman,0,...,0,0,0,0,1,0,1,NaN,NaN,NaN
136095,575,2,Kolkata Knight Riders,Sunrisers Hyderabad,19,2,JO Holder,R Sathish,Mustafizur Rahman,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


### Most Dismissals

In [85]:
dismissed_df = delivery_df[delivery_df['player_dismissed'].notnull()]

batsman_bowler_pairs = Counter()
for index, row in dismissed_df[['bowler', 'player_dismissed']].iterrows():
    batsman_bowler_pairs['{}:{}'.format(row['bowler'], row['player_dismissed'])] += 1

In [96]:
most_dismissed_pairs = batsman_bowler_pairs.most_common(30)
result = []

for p in most_dismissed_pairs:
    bowler, batsman = p[0].split(':')
    result.append({
        'source': bowler,
        'target': batsman,
        'value': p[1]
    })
    
print('bowlers', set(map(lambda r: r['source'], result)))
print('\n')
print('batsmen', set(map(lambda r: r['target'], result)))
print('\n')
print(result)
print('\n')
print(set(map(lambda r: r['source'], result)).intersection(set(map(lambda r: r['target'], result))))

bowlers {'A Nehra', 'MM Sharma', 'R Sharma', 'P Kumar', 'RA Jadeja', 'RP Singh', 'A Mishra', 'PP Chawla', 'PP Ojha', 'B Kumar', 'R Vinay Kumar', 'DJ Bravo', 'Z Khan', 'SK Trivedi', 'Harbhajan Singh', 'IK Pathan'}


batsmen {'G Gambhir', 'SC Ganguly', 'JH Kallis', 'R Dravid', 'DJ Hussey', 'KC Sangakkara', 'BB McCullum', 'MEK Hussey', 'MS Dhoni', 'AB de Villiers', 'SR Watson', 'PA Patel', 'AC Gilchrist', 'RV Uthappa', 'AM Rahane', 'SK Raina', 'RA Jadeja', 'V Kohli', 'RG Sharma', 'NV Ojha', 'MK Pandey', 'AT Rayudu'}


[{'source': 'Z Khan', 'target': 'MS Dhoni', 'value': 7}, {'source': 'PP Ojha', 'target': 'MS Dhoni', 'value': 7}, {'source': 'A Nehra', 'target': 'V Kohli', 'value': 6}, {'source': 'R Vinay Kumar', 'target': 'RG Sharma', 'value': 6}, {'source': 'B Kumar', 'target': 'AM Rahane', 'value': 6}, {'source': 'Z Khan', 'target': 'G Gambhir', 'value': 5}, {'source': 'SK Trivedi', 'target': 'RG Sharma', 'value': 5}, {'source': 'PP Ojha', 'target': 'G Gambhir', 'value': 5}, {'source': 

In [100]:
batsmen = {'G Gambhir', 'SC Ganguly', 'JH Kallis', 'R Dravid', 'DJ Hussey', 'KC Sangakkara', 'BB McCullum', 'MEK Hussey', 'MS Dhoni', 'AB de Villiers', 'SR Watson', 'PA Patel', 'AC Gilchrist', 'RV Uthappa', 'AM Rahane', 'SK Raina', 'RA Jadeja', 'V Kohli', 'RG Sharma', 'NV Ojha', 'MK Pandey', 'AT Rayudu'}

In [101]:
d = {}
for b in batsmen:
    d[b]='batsmen'

In [102]:
d

{'AB de Villiers': 'batsmen',
 'AC Gilchrist': 'batsmen',
 'AM Rahane': 'batsmen',
 'AT Rayudu': 'batsmen',
 'BB McCullum': 'batsmen',
 'DJ Hussey': 'batsmen',
 'G Gambhir': 'batsmen',
 'JH Kallis': 'batsmen',
 'KC Sangakkara': 'batsmen',
 'MEK Hussey': 'batsmen',
 'MK Pandey': 'batsmen',
 'MS Dhoni': 'batsmen',
 'NV Ojha': 'batsmen',
 'PA Patel': 'batsmen',
 'R Dravid': 'batsmen',
 'RA Jadeja': 'batsmen',
 'RG Sharma': 'batsmen',
 'RV Uthappa': 'batsmen',
 'SC Ganguly': 'batsmen',
 'SK Raina': 'batsmen',
 'SR Watson': 'batsmen',
 'V Kohli': 'batsmen'}